# 約束事  

加速度の名前は例として，Participants07-First-01-win=0256-sld=0032  
下のセルの変更するところは，
* Version
* Time
* CommonName

In [1]:
import numpy as np
import datetime
import os
import copy
# Generate WindowFrame Function
from mymodule import window
import joblib
import pandas as pd

from tqdm import tqdm_notebook

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Define Data Name
DATE= str( datetime.date.today() )+'/'

# Parametor about " Data Path"
StorePath = "/media/takeyama/Transfer/02_ActivityResearch/"

ParticipantsNum="Participants07/"
Version = "01_First/"
Time = "02_Second/"
CommonName = 'Participants07-First-02'
LabelName = CommonName+'.csv'

TITLE = ParticipantsNum+Version

DataPath = StorePath+ParticipantsNum+Version+"RawData/"+Time

if not os.path.exists(StorePath+TITLE+'LabelData/'+Time): 
    os.makedirs(StorePath+TITLE+'LabelData/'+Time)
LabelDataPath=StorePath+TITLE+'LabelData/'+Time

if not os.path.exists(StorePath+TITLE+'dictionary/'+Time): 
    os.makedirs(StorePath+TITLE+'dictionary/'+Time)
DictionaryDataPath=StorePath+TITLE+'dictionary/'+Time

if not os.path.exists(StorePath+TITLE+'feature/'+Time): 
    os.makedirs(StorePath+TITLE+'feature/'+Time)
FeaturePath=StorePath+TITLE+'feature/'+Time

if not os.path.exists(StorePath+TITLE+'window/'+Time): 
    os.makedirs(StorePath+TITLE+'window/'+Time)
WindowDataPath=StorePath+TITLE+'window/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/'+Time)
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/'+Time)
ParametorPath=StorePath+TITLE+DATE+'parametor/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'result/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'result/'+Time)
ResultPath=StorePath+TITLE+DATE+'result/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'graph/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'graph/'+Time)
GraphPath=StorePath+TITLE+DATE+'graph/'+Time

# Task01 Read CSV Data

In [2]:
# センサデータの読み込み
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
    
    return FilePath

In [3]:
SensorDataFileNameList(DataPath)

['mem-USB14011717-20161019-132642987.csv',
 'mem-USB14011718-20161019-132642947.csv',
 'mem-USB15082037-20161019-132643904.csv',
 'mem-USB15082038-20161019-132643877.csv',
 'mem-USB15082039-20161019-132643809.csv',
 'mem-USB15082040-20161019-132643804.csv']

# Task02 Acc&Agr scale change  
Acc: [0.1 mG] -> [G]  
Gyr: [0.01 dps] -> [dps]  

In [4]:
# CSVを読み込んで，CSVの中身を辞書型で表現する．
def ImportCSV(csv_file,SensorName,mode='Round'):
    import pandas as pd
    '''
    ~Argument~
    csv_file -> ファイル名 
    mode Round -> 四捨五入
         Roundup -> 切り上げ
         Rounddown -> 切り捨て
    
    ~Conversion~
    Acc Data  [0.1mG]=>[G]
    Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
    '''
    # data dictionary 
    RawData={}   
    AccConversion = 0.1 * 0.001
    GyrConversion = 0.01
    
    # design dataframe and import csv
    data = pd.read_csv(csv_file)
    data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
    data = data[ data['Type']=='ags']

    # convert numpy.darray 

    AccX=data.AccX.values*AccConversion
    AccY=data.AccY.values*AccConversion
    AccZ=data.AccZ.values*AccConversion
    
    GyrX=data.GyrX.values*GyrConversion
    GyrY=data.GyrY.values*GyrConversion
    GyrZ=data.GyrZ.values*GyrConversion

    # regist each raw data 
    RawData['AccX'] = AccX
    RawData['AccY'] = AccY
    RawData['AccZ'] = AccZ
    RawData['GyrX'] = GyrX
    RawData['GyrY'] = GyrY
    RawData['GyrZ'] = GyrZ
    RawData['Name'] = SensorName

    RawData['Time'] = data.Time.values
    return RawData

In [5]:
Label = ['01_LeftHand','02_RightHand','03_LeftLeg','04_Rightleg','05_West','06_Chest']
DataFileNameList =SensorDataFileNameList(DataPath)

DataList = list()

for name,l in zip( DataFileNameList,Label):
    DataList.append( ImportCSV(DataPath+name,l) )

# TASK03 Sync Each Senser's Length

In [6]:
def MakeCommonSection(inputDataArray):
    """
    inputDataArray: センサデータの配列
    
    return
        TrimDataArray: スタート時間とゴール時間を整理したデータ
        startTime: TrimDataArrayのデータ
        goalTime: TrimDataArrayのゴール時間
    
    """
    # 共通区間のスタート時間、ゴール時間を求める
    # スタート時間を求める
    startTime = min(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if startTime < min(inputDataArray[i]['Time']):
            startTime = min(inputDataArray[i]['Time'])
    
    # ゴール時間を求める
    goalTime = max(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if goalTime > max(inputDataArray[i]['Time']):
            goalTime = max(inputDataArray[i]['Time'])

    # 共通区間のスタート時間のインデックス、ゴール時間のインデックスを探索する
    def CalcSearchIndexFromTime(data, keyTime):
        """
        data　辞書型
        keyTime data['Time']の中の探す値
        """
        count = 0
        for i in range(0, len(data['Time'])):
            if keyTime == data['Time'][i]:
                print str(keyTime)+' is much in the index  whose number is '+str(i)
                return i
            
    startIndex = np.array([])
    goalIndex = np.array([])
    for obj in inputDataArray:
        #print 'start'
        startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
        #print 'goal'
        goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

    tmp={}
    TrimDataArray =[]

    # センサデータすべて（時刻、加速度、角速度）に対して共通区間のみのデータを抽出
    for number,iDA in enumerate( inputDataArray ):

        tmp['AccX'] = copy.deepcopy( iDA['AccX'][startIndex[number]:goalIndex[number]] )
        tmp['AccY'] = copy.deepcopy( iDA['AccY'][startIndex[number]:goalIndex[number]] )
        tmp['AccZ'] = copy.deepcopy( iDA['AccZ'][startIndex[number]:goalIndex[number]] )
        tmp['GyrX'] = copy.deepcopy( iDA['GyrX'][startIndex[number]:goalIndex[number]] )
        tmp['GyrY'] = copy.deepcopy( iDA['GyrY'][startIndex[number]:goalIndex[number]] )
        tmp['GyrZ'] = copy.deepcopy( iDA['GyrZ'][startIndex[number]:goalIndex[number]] )
        tmp['Time'] = copy.deepcopy( iDA['Time'][startIndex[number]:goalIndex[number]] )
        tmp['Name'] = copy.deepcopy( iDA['Name'] )
        TrimDataArray.append(copy.deepcopy(tmp) )    
       
    return TrimDataArray,startTime,goalTime

In [7]:
TrimDataArray,StartTime,GoalTime = MakeCommonSection(DataList)

48404019 is much in the index  whose number is 917
49461928 is much in the index  whose number is 1058826
48404019 is much in the index  whose number is 957
49461928 is much in the index  whose number is 1058866
48404019 is much in the index  whose number is 0
49461928 is much in the index  whose number is 1057909
48404019 is much in the index  whose number is 27
49461928 is much in the index  whose number is 1057936
48404019 is much in the index  whose number is 95
49461928 is much in the index  whose number is 1058004
48404019 is much in the index  whose number is 100
49461928 is much in the index  whose number is 1058009


In [8]:
for i in range(6):
    np.savez(DictionaryDataPath+TrimDataArray[i]['Name'],data=TrimDataArray[i])

# TASK 04 Read LabelData and Make TimeDomain Data

In [9]:
def MakeTimeDomainLabelData(DataName,StartTime,GoalTime):
    # import LabelData -> pandas
    LabelDf = pd.read_csv(DataName,encoding='Shift-JIS')
    LabelData = LabelDf.as_matrix() #pandas -> ndarray
    
    # timedata hh.mm.ss -> ms
    TimeArray = pd.to_datetime( LabelDf['#Time'].values ) 
    t = np.array([ TimeArray[i].hour*3600*1000+TimeArray[i].minute*60*1000+TimeArray[i].second*1000+TimeArray[i].microsecond*0.001
                 for i in range( len(TimeArray) )]).astype(int)
    
    LabelData[:,0] = t.astype(int) #time data: unicode -> int
    # Time Series
    LabelArray = np.array([
        np.append( np.array(LabelData[i,0]+l) , LabelData[i,1:])
        for i in range(LabelData.shape[0]-1) 
        for l in range(LabelData[i+1,0]-LabelData[i,0])
    ])
    
    StartLabelArray = np.array([
        np.append( StartTime+i, LabelArray[0,1:])
        for i in range(LabelArray[0,0]-StartTime)
    ])
    
    GoalLabelArray = np.array([
        np.append( GoalTime+i, LabelArray[-1,1:])
        for i in range(1,1+GoalTime-LabelArray[-1,0])
    ])
    
    LabelArrayComp = np.vstack((StartLabelArray, LabelArray))
    LabelArrayComp = np.vstack((LabelArrayComp, GoalLabelArray))
    df = pd.DataFrame(LabelArrayComp)
    df.columns = LabelDf.columns
    
    return df.to_dict()

In [10]:
dictLabel=MakeTimeDomainLabelData(LabelDataPath+LabelName,StartTime,GoalTime)
np.savez(DictionaryDataPath+'07_LabelData',data=dictLabel)

In [11]:
dictLabel.keys()

[u'RightHand-Verb',
 u'#Time',
 u'LeftHand-Object',
 u'West',
 u'LeftHand-Verb',
 u'Leg-Number',
 u'ActiveNumber',
 u'Leg-Object',
 u'Leg-Verb',
 u'RightHand-Object']

In [12]:
df = pd.DataFrame(dictLabel)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057910 entries, 0 to 1057909
Data columns (total 10 columns):
#Time               1057910 non-null int64
ActiveNumber        1057910 non-null int64
LeftHand-Object     1057910 non-null object
LeftHand-Verb       1057910 non-null object
Leg-Number          1057910 non-null object
Leg-Object          1057910 non-null object
Leg-Verb            1057910 non-null object
RightHand-Object    1057910 non-null object
RightHand-Verb      1057910 non-null object
West                1057910 non-null object
dtypes: int64(2), object(8)
memory usage: 88.8+ MB


In [14]:
print "Each columns  has some unique label"
for c in df.columns:
    print 'columns='+c+' unique='+str( len( df[c].unique() ) )

Each columns  has some unique label
columns=#Time unique=1057910
columns=ActiveNumber unique=5
columns=LeftHand-Object unique=23
columns=LeftHand-Verb unique=18
columns=Leg-Number unique=1
columns=Leg-Object unique=1
columns=Leg-Verb unique=3
columns=RightHand-Object unique=24
columns=RightHand-Verb unique=19
columns=West unique=3


In [15]:
print "Each ActiveNumber has some label"
for key in df['ActiveNumber'].unique():
    print 'Columns='+str(key)+' count='+str(df[df['ActiveNumber']==key].shape[0] )

Each ActiveNumber has some label
Columns=200 count=21438
Columns=201 count=708885
Columns=202 count=14328
Columns=203 count=275913
Columns=204 count=37346
